In [1]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=10)

from grizly.scheduling.registry import RedisDB, Trigger, Job

## RedisDB

In [2]:
rdb = RedisDB()

2020-08-19 14:06:30,584 - grizly.config - DEBUG - Config data loaded from /home/analyst/.grizly/config.json.


In [3]:
rdb.get_jobs()

[]

In [4]:
rdb.get_triggers()

[]

In [5]:
rdb.get_job_runs()

[]

## Job

In [6]:
import dask
import logging
from datetime import datetime

logger = logging.getLogger("distributed.worker.printing_time")
@dask.delayed
def print_time():
    logger.info(f"Hello, it's {datetime.now().__str__()}")
    
time_log = print_time()

@dask.delayed
def add(x, y):
    return x + y

_sum = add(1, 2)

In [7]:
job1 = Job("test_print")
job1.register(tasks=[time_log], crons="* * * * *", if_exists="replace")

job2 = Job("test_sum")
job2.register(tasks=[_sum], upstream="test_print", if_exists="replace")

2020-08-19 14:06:30,822 - grizly.scheduling.registry - DEBUG - Job test_print cron * * * * * has been added to rq sheduler with id 556cc75d-4dea-4d56-bc83-7b887cbd76ef
2020-08-19 14:06:30,824 - grizly.scheduling.registry - INFO - Job has been added to the scheduler
2020-08-19 14:06:30,835 - grizly.scheduling.registry - INFO - Job test_print successfully registered
2020-08-19 14:06:30,863 - grizly.scheduling.registry - INFO - Adding ['test_sum'] to downstream...
2020-08-19 14:06:30,871 - grizly.scheduling.registry - INFO - Job test_sum successfully registered


Job(name='test_sum')

In [8]:
job1.info()

name: test_print
owner: None
crons: ['* * * * *']
downstream: [Job(name='test_sum')]
upstream: []
triggers: []


In [9]:
job1.runs

[JobRun(job_name='test_print, id='1')]

In [10]:
job2.info()

name: test_sum
owner: None
crons: []
downstream: []
upstream: [Job(name='test_print')]
triggers: []


In [11]:
job2.runs

[]

In [12]:
job1.submit(to_dask=False)

2020-08-19 14:06:31,140 - grizly.scheduling.registry - INFO - Submitting job test_print...
distributed.worker.printing_time - INFO - Hello, it's 2020-08-19 14:06:31.174257
2020-08-19 14:06:31,178 - grizly.scheduling.registry - WARNING - Submitting downstream_jobs...
2020-08-19 14:06:31,200 - grizly.scheduling.registry - INFO - Job test_print finished with status success


[None]

In [13]:
job1.runs

[JobRun(job_name='test_print, id='1')]

In [14]:
job2.runs

[]

In [15]:
job1.unregister()

2020-08-19 14:06:31,334 - grizly.scheduling.registry - DEBUG - Rq job 556cc75d-4dea-4d56-bc83-7b887cbd76ef removed from the scheduler
2020-08-19 14:06:31,347 - grizly.scheduling.registry - INFO - Job has been removed from the scheduler
2020-08-19 14:06:31,413 - grizly.scheduling.registry - INFO - Removing ['test_print'] from upstream...
2020-08-19 14:06:31,429 - grizly.scheduling.registry - INFO - Removing ['test_sum'] from downstream...
2020-08-19 14:06:31,452 - grizly.scheduling.registry - INFO - Job test_print successfully removed from registry


In [16]:
job2.unregister()

2020-08-19 14:06:31,503 - grizly.scheduling.registry - INFO - Job test_sum successfully removed from registry


## ADMIN

#### Unregister all jobs

#### Remove all keys from Redis

#### Cancel all rq scheduler jobs